In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import sqlite3
from collections import deque

In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
seed_page = "https://en.wikipedia.org"
start_page = "/wiki/Mathematics"

In [4]:
def create_cursor():
    conn = sqlite3.connect('../data/wiki_crawler.sqlite')
    cur = conn.cursor()    
    cur.execute('''
                CREATE TABLE IF NOT EXISTS WikiPages
                (id INTEGER PRIMARY KEY, title TEXT, URL TEXT UNIQUE, retrived BOOLEAN, citedCount INTEGER)
                ''')
    cur.execute("""
                CREATE TABLE IF NOT EXISTS Cites
                (from_id INTEGER, to_id INTEGER, UNIQUE(from_id, to_id))
                """)
    return conn, cur

In [5]:
conn, cur = create_cursor()

In [14]:
def crawl_with_given_page(conn, cur, start_page = "/wiki/Mathematics", time = 100, seed_page = "https://en.wikipedia.org"):
    cur.execute('SELECT retrived from WikiPages WHERE URL = ? LIMIT 1', (start_page, ))
    try:
        retrived = cur.fetchone()[0]
        if retrived == 1:
            print("The given start_page has been crawled already.")
            return
        print("Start crawling")
    except:
        print("Start crawling")
        
    queue = deque([start_page])
    count = 0
    total_count = 0
    while queue and count<time:
        # print(count)
        url = queue.popleft()

        cur.execute('SELECT retrived from WikiPages WHERE URL = ? LIMIT 1', (url, ))
        try:
            retrived = cur.fetchone()[0]
            if retrived == 1:
                # print("The given website has been crawled already.")
                continue
            cur.execute('UPDATE WikiPages SET retrived = ? WHERE URL = ?', (1, url))
        except:
            total_count += 1
            cur.execute('INSERT INTO WikiPages (URL, retrived, citedCount) VALUES (?, 1, 0)', (url, ))

        page_count = crawl_one_page(url, cur, adding = True, queue = queue)
        
        if page_count>-1:    
            count += 1
            total_count += page_count
            if count%100==0:
                print('count: ', count)
                conn.commit()
    
    conn.commit()
    print("Total new pages crawled: ", count)
    print("Total new pages found: ", total_count)
    return

In [15]:
def crawl_fetch_30_candidates(queue, cur):
    cur.execute('SELECT URL from WikiPages WHERE retrived = ? ORDER BY citedCount DESC LIMIT 30', (0, ))
    for row in cur:
        queue.append(row[0])    
        
def crawl_one_page(url, cur, adding = False, queue = None, seed_page = "https://en.wikipedia.org"):
    cur.execute("SELECT id FROM WikiPages WHERE url = ?", (url, ))
    from_id = cur.fetchone()[0]
    try:
        html = urllib.request.urlopen(seed_page+url, context=ctx).read()
    except:
        # print(seed_page+url, " can't be retrived.")
        return -1   
    count = 0
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.title.string
    # print(title)
    cur.execute('UPDATE WikiPages SET title = ? WHERE URL = ?', (title, url))
    # Retrieve all of the anchor tags
    tags = soup('a')
    cited = set()
    for tag in tags:
        to_url = tag.get('href', None)
        if not to_url or not to_url.startswith('/') or to_url.startswith('//') or to_url in cited:
            continue
        cited.add(to_url)
        cur.execute('SELECT citedCount, id from WikiPages WHERE URL = ? LIMIT 1', (to_url, ))
        if adding:
            queue.append(to_url)
        try:
            num = cur.fetchone()[0]
            cur.execute('UPDATE WikiPages SET citedCount = ? WHERE URL = ?', (num+1, to_url))
        except:
            count += 1
            cur.execute('INSERT INTO WikiPages (URL, retrived, citedCount) VALUES (?, 0, 1)', (to_url, ))
        cur.execute('SELECT id FROM WikiPages WHERE url = ?', (to_url, ))
        to_id = cur.fetchone()[0]
        if from_id!=to_id:
            cur.execute('INSERT OR IGNORE INTO Cites (from_id, to_id) VALUES (?, ?)', (from_id, to_id))
    return count

def crawl_with_existing_page(conn, cur, time = 100, seed_page = "https://en.wikipedia.org"):
    queue = deque()
    crawl_fetch_30_candidates(queue, cur)
    if len(queue)==0:
        print('No avaiable link in the database to crawl')
        return
    else:
        print('Start crawling')
        
    count = 0
    total_count = 0
    while count<time:
        # print(count)
        if len(queue)==0:
            crawl_fetch_30_candidates(queue, cur)
            if len(queue)==0:
                print('No available links')
                break
                       
        url = queue.popleft()
        cur.execute('UPDATE WikiPages SET retrived = ? WHERE URL = ?', (1, url))
        
        page_count = crawl_one_page(url, cur)
 
        if page_count>-1:    
            count += 1
            total_count += page_count
            if count%100==0:
                print('count: ', count)
                conn.commit()
    
    conn.commit()
    print("Total new pages crawled: ", count)
    print("Total new pages found: ", total_count)
    return

In [8]:
crawl_with_given_page(conn, cur, time = 2000)

The given start_page has been crawled already.


In [9]:
crawl_with_given_page(conn, cur, start_page = "/wiki/Mathematics", time = 400)

The given start_page has been crawled already.


In [19]:
crawl_with_given_page(conn, cur, start_page = "/wiki/Almost_periodic_function", time = 300)

Start crawling
count:  100
count:  200
count:  300
Total new pages crawled:  300
Total new pages found:  23903


In [16]:
crawl_with_existing_page(conn, cur, 500)

Start crawling
count:  100
count:  200
count:  300
count:  400
count:  500
Total new pages crawled:  500
Total new pages found:  34970


In [20]:
cur.execute('SELECT title, url, citedCount FROM WikiPages ORDER BY citedCOunt DESC LIMIT 30')

In [21]:
for row in cur:
    print(row)

('User talk:2601:641:500:5CC0:9026:D008:E47C:BA3B - Wikipedia', '/wiki/Special:MyTalk', 9800)
('User contributions for 2601:641:500:5CC0:9026:D008:E47C:BA3B - Wikipedia', '/wiki/Special:MyContributions', 9800)
('Wikipedia, the free encyclopedia', '/wiki/Main_Page', 9800)
('Portal:Contents - Wikipedia', '/wiki/Portal:Contents', 9800)
('Portal:Featured content - Wikipedia', '/wiki/Portal:Featured_content', 9800)
('Portal:Current events - Wikipedia', '/wiki/Portal:Current_events', 9800)
('Speed limits in Romania - Wikipedia', '/wiki/Special:Random', 9800)
('Help:Contents - Wikipedia', '/wiki/Help:Contents', 9800)
('Wikipedia:About - Wikipedia', '/wiki/Wikipedia:About', 9800)
('Wikipedia:Community portal - Wikipedia', '/wiki/Wikipedia:Community_portal', 9800)
('Recent changes - Wikipedia', '/wiki/Special:RecentChanges', 9800)
('Wikipedia:File Upload Wizard - Wikipedia', '/wiki/Wikipedia:File_Upload_Wizard', 9800)
('Special pages - Wikipedia', '/wiki/Special:SpecialPages', 9800)
('Wikipedia

In [22]:
# Get all cited urls from the given source
def get_cited_urls_from_source(cur, from_url):
    cur.execute("SELECT id FROM WikiPages WHERE URL = ? LIMIT 1", (from_url,))
    try:
        source = cur.fetchone()[0]
    except:
        print("The given url is not found!")
        return []
    res = []
    cur.execute("""
                SELECT URL, id from WikiPages JOIN Cites ON WikiPages.id = Cites.to_id
                WHERE Cites.from_id = ?
                """, (source, ))
    for row in cur:
        res.append(row)
    print("The given page cites %d different internal pages." % (len(res)))
    return res

# Get all urls that cite the given destination
def get_citing_urls_to_destination(cur, to_url):
    cur.execute("SELECT id FROM WikiPages WHERE URL = ? LIMIT 1", (to_url, ))
    try:
        dest = cur.fetchone()[0]
    except:
        print("The given url is not found!")
        return []
    res = []
    cur.execute("""
                SELECT URL, id from WikiPages JOIN Cites ON WikiPages.id = Cites.from_id
                WHERE Cites.to_id = ?
                """, (dest, ))
    for row in cur:
        res.append(row)
    print("There are %d pages found that cites the given link" % (len(res)))
    return res

In [23]:
urls = get_cited_urls_from_source(cur, "/wiki/Toeplitz_operator")
for i in range(10):
    print(urls[i])

The given page cites 69 different internal pages.
('/wiki/Mathematical_analysis', 18)
('/wiki/Continuous_function', 244)
('/wiki/International_Standard_Book_Number', 401)
('/wiki/Help:Category', 490)
('/wiki/Special:MyTalk', 517)
('/wiki/Special:MyContributions', 518)
('/wiki/Main_Page', 524)
('/wiki/Portal:Contents', 525)
('/wiki/Portal:Featured_content', 526)
('/wiki/Portal:Current_events', 527)


In [24]:
urls = get_citing_urls_to_destination(cur, "/wiki/Mathematics")
for i in range(10):
    print(urls[i])

There are 755 pages found that cites the given link
('/wiki/Mathematics_(disambiguation)', 3)
('/wiki/Math_(disambiguation)', 4)
('/wiki/File:Euclid.jpg', 5)
('/wiki/Euclid', 6)
('/wiki/Mathematical_structure', 13)
('/wiki/Algebra', 14)
('/wiki/Geometry', 16)
('/wiki/Calculus', 17)
('/wiki/Mathematical_analysis', 18)
('/wiki/Definition', 19)


In [25]:
def get_most_cites(cur, num = 20):
    cur.execute('SELECT URL, title, citedCount FROM WikiPages ORDER BY citedCount DESC LIMIT ?', (num, ))
    res = []
    try:
        for row in cur:
            res.append(row)
    except:
        print('No available links in WikiPages')
    return res
    

In [26]:
urls = get_most_cites(cur, 30)
for row in urls:
    print(row)

('/wiki/Special:MyTalk', 'User talk:2601:641:500:5CC0:9026:D008:E47C:BA3B - Wikipedia', 9800)
('/wiki/Special:MyContributions', 'User contributions for 2601:641:500:5CC0:9026:D008:E47C:BA3B - Wikipedia', 9800)
('/wiki/Main_Page', 'Wikipedia, the free encyclopedia', 9800)
('/wiki/Portal:Contents', 'Portal:Contents - Wikipedia', 9800)
('/wiki/Portal:Featured_content', 'Portal:Featured content - Wikipedia', 9800)
('/wiki/Portal:Current_events', 'Portal:Current events - Wikipedia', 9800)
('/wiki/Special:Random', 'Speed limits in Romania - Wikipedia', 9800)
('/wiki/Help:Contents', 'Help:Contents - Wikipedia', 9800)
('/wiki/Wikipedia:About', 'Wikipedia:About - Wikipedia', 9800)
('/wiki/Wikipedia:Community_portal', 'Wikipedia:Community portal - Wikipedia', 9800)
('/wiki/Special:RecentChanges', 'Recent changes - Wikipedia', 9800)
('/wiki/Wikipedia:File_Upload_Wizard', 'Wikipedia:File Upload Wizard - Wikipedia', 9800)
('/wiki/Special:SpecialPages', 'Special pages - Wikipedia', 9800)
('/wiki/Wik

In [27]:
def get_title_with_keyword(keyword, cur):
    cur.execute("SELECT URL, title FROM WikiPages WHERE title LIKE ?", ('%'+keyword+'%', ))
    res = []
    try:
        for row in cur:
            res.append(row)
        print("There are %d titles with given keyword." % (len(res)))
    except:
        print("No such title")
    return res

In [28]:
urls = get_title_with_keyword("Operator", cur)
for row in urls:
    print(row)

There are 25 titles with given keyword.
('/wiki/Operator_theory', 'Operator theory - Wikipedia')
('/wiki/Operator_(mathematics)', 'Operator (mathematics) - Wikipedia')
('/wiki/Self-adjoint_operator', 'Self-adjoint operator - Wikipedia')
('/wiki/Compact_operator', 'Compact operator - Wikipedia')
('/wiki/Multiplication_operator', 'Multiplication operator - Wikipedia')
('/wiki/Toeplitz_operator', 'Toeplitz operator - Wikipedia')
('/wiki/Dilation_(operator_theory)', 'Dilation (operator theory) - Wikipedia')
('/w/index.php?title=Toeplitz_operator&action=edit&section=1', 'Editing Toeplitz operator (section) - Wikipedia')
('/w/index.php?title=Toeplitz_operator&action=edit&section=2', 'Editing Toeplitz operator (section) - Wikipedia')
('/w/index.php?title=Toeplitz_operator&action=edit&section=3', 'Editing Toeplitz operator (section) - Wikipedia')
('/wiki/Category:Operator_theory', 'Category:Operator theory - Wikipedia')
('/wiki/Talk:Toeplitz_operator', 'Talk:Toeplitz operator - Wikipedia')
('/

In [29]:
def retrived_page_ratio(cur):
    cur.execute('SELECT COUNT(1), retrived FROM WikiPages GROUP BY retrived')
    uncrawled, crawled = cur
    print("Total number of pages in the dataset: ", uncrawled[0]+crawled[0])
    # print("Total number of pages crawled: ", crawled[0])
    print("Crawled ratio: %5.3f" % (crawled[0]/(uncrawled[0]+crawled[0])))

In [30]:
retrived_page_ratio(cur)

Total number of pages in the dataset:  1058687
Crawled ratio: 0.009
